In [1]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우의 한글 폰트
plt.rcParams['axes.unicode_minus'] = False       # 마이너스 기호 깨짐 방지

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 처리
import pandas as pd
import numpy as np

train_df = pd.read_csv("/data/train.csv")
test_df = pd.read_csv("/data/test.csv")

# "UID" 컬럼 유지 (제출 파일용)
test_uid = test_df[["UID"]]

# "UID" 컬럼 삭제
train_df.drop(columns=["UID"], inplace=True)
test_df.drop(columns=["UID"], inplace=True)

# X, y 분리
X = train_df.drop(columns=["채무 불이행 여부"])
y = train_df["채무 불이행 여부"]

In [3]:
def convert_years(val):
    if val == '1년 미만':
        return 0
    elif val == '10년 이상':
        return 10
    else:
        # '4년', '6년' 등에서 '년'을 제거하고 정수형으로 변환
        return int(val.replace('년', ''))

# train, test 데이터에 대해 변환 적용
X['현재 직장 근속 연수'] = X['현재 직장 근속 연수'].apply(convert_years)
test_df['현재 직장 근속 연수'] = test_df['현재 직장 근속 연수'].apply(convert_years)

# 변환 결과 확인
print(X['현재 직장 근속 연수'].unique())


[10  4  6  2  8  3  5  9  7  0  1]


In [4]:
# 원-핫 인코딩 적용
X = pd.get_dummies(X, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)
test_df = pd.get_dummies(test_df, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)

In [5]:
# 로그 변환
log_columns = ["현재 미상환 신용액", "월 상환 부채액", "현재 대출 잔액", "연간 소득"] # 최대 신용한도 컬럼도 추가 가능
for col in log_columns:
    X[col] = np.log1p(X[col])
    test_df[col] = np.log1p(test_df[col])

# "마지막 연체 이후 경과 개월 수"가 0이면 "연체 없음" 컬럼 추가
X["연체 없음"] = (X["마지막 연체 이후 경과 개월 수"] == 0).astype(int)
test_df["연체 없음"] = (test_df["마지막 연체 이후 경과 개월 수"] == 0).astype(int)

In [6]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)
test_imputed = imputer.transform(test_df)
print(X.shape)
print(X_imputed.shape)
print(test_df.shape)
print(test_imputed.shape)

(10000, 31)
(10000, 31)
(2062, 31)
(2062, 31)


In [7]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_imputed)
# test_scaled = scaler.transform(test_imputed)

In [8]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 스케일링 + 로지스틱 회귀 파이프라인
log_reg = Pipeline([
    ('scaler', StandardScaler()),  # 표준화
    ('classifier', LogisticRegression(class_weight='balanced', random_state=42))
])

# 모델 학습
log_reg.fit(X_imputed, y)

# 로지스틱 회귀를 사용해 테스트 데이터 예측
test_pred = log_reg.predict_proba(test_df)[:, 1]  # 양성 클래스(채무 불이행)의 확률
print(test_pred.shape)
# 제출 파일 생성
submission = pd.DataFrame({
    'UID': test_uid.ravel(),
    '채무 불이행 확률': test_pred
})

# CSV 파일 저장
submission.to_csv('/data/submission.csv', index=False)
print("✅ 'submission.csv' 파일이 '/data' 경로에 저장되었습니다.")


(2062,)
✅ 'submission.csv' 파일이 '/data' 경로에 저장되었습니다.
